# Creating a date dimension table

In [1]:
# importing data analysis library
import pandas as pd
import numpy as np

# first column
# creating the base range
time_range = pd.date_range(start= '00:00:00',
                          end= '23:59:59',
                          freq= 'min')

# range as df with format dd-mm-yyyy
time_dim = pd.DataFrame(data= time_range,
                        columns= ['ISO_time'])
time_dim['ISO_time'] = time_dim['ISO_time'].dt.strftime('%H:%M:%S')

# displaying the df so far
time_dim

,ISO_time
0,00:00:00
1,00:01:00
2,00:02:00
3,00:03:00
4,00:04:00
...,...
1435,23:55:00
1436,23:56:00
1437,23:57:00
1438,23:58:00


In [2]:
# extracting features from the time
time_dim['AM-PM_string'] = time_dim['ISO_time'].apply(lambda x:
                                    pd.to_datetime(
                                     x, format= '%H:%M:%S')
                                    .strftime('%I:%M %p'))
time_dim['24h_string'] = time_dim['ISO_time'].apply(lambda x: 
                                    pd.to_datetime(
                                    x, format='%H:%M:%S')
                                    .strftime('%Hh%M'))
time_dim['Military_Time'] = time_dim['ISO_time'].apply(lambda x: 
                                   pd.to_datetime(
                                   x, format='%H:%M:%S'
                                   ).strftime('%H%M'))
time_dim['hour'] = time_dim['ISO_time'].apply(lambda x: 
                                          pd.to_datetime(x, 
                                          format= '%H:%M:%S')
                                          .hour)
time_dim['minute'] = time_dim['ISO_time'].apply(lambda x:
                                            pd.to_datetime(
                                            x, format= '%H:%M:%S')
                                            .minute)
time_dim['AM_PM'] = time_dim['ISO_time'].apply(lambda x:
                                               pd.to_datetime(
                                                x, format= '%H:%M:%S')
                                               .strftime('%p'))
time_dim['min_of_day'] = time_dim['hour'] * 60 + time_dim['minute']
time_dim['sec_of_day'] = time_dim['min_of_day'] * 60 \
                            + time_dim['ISO_time'].apply(lambda x:
                                           pd.to_datetime(x,
                                           format= '%H:%M:%S')
                                           .second)
time_dim['QuarterHour'] = time_dim['minute'] // 15 + 1
time_dim['TimePeriod'] = time_dim['hour'].apply(lambda x:
                            'Morning' if 5 <= x < 12 
                            else 'Afternoon' if 12 <= x < 17 
                            else 'Evening' if 17 <= x < 21
                            else 'Night')
time_dim['Shift'] = time_dim['hour'].apply(lambda x:
                            'Morning Shift' if 6 <= x < 14
                            else 'Evening Shift' if 14 <= x < 22
                            else 'Night Shift')
time_dim['IsWorkHour'] = time_dim['hour'].apply(lambda x:
                                                9 <= x <= 17)


time_dim.sample(5)

,ISO_time,AM-PM_string,24h_string,Military_Time,hour,minute,AM_PM,min_of_day,sec_of_day,QuarterHour,TimePeriod,Shift,IsWorkHour
1082,18:02:00,06:02 PM,18h02,1802,18,2,PM,1082,64920,1,Evening,Evening Shift,False
1007,16:47:00,04:47 PM,16h47,1647,16,47,PM,1007,60420,4,Afternoon,Evening Shift,True
550,09:10:00,09:10 AM,09h10,0910,9,10,AM,550,33000,1,Morning,Morning Shift,True
391,06:31:00,06:31 AM,06h31,0631,6,31,AM,391,23460,3,Morning,Morning Shift,False
879,14:39:00,02:39 PM,14h39,1439,14,39,PM,879,52740,3,Afternoon,Evening Shift,True


## Exporting to .csv for use in other data modelling software

In [3]:
time_dim.to_csv('time_dim.csv', index= False)